In [7]:
import numpy as np
import time
import cProfile

# print(np.vectorize(np.binary_repr)(bit))

In [16]:
def handeval(yourhand):
   
    mask = np.array([0x2000,0x4000,0x8000,0x10000]).reshape(4,1,1)
    broadcast = np.copy(np.tile(yourhand,(4,1,1)))
    suitnum = (np.sum((( broadcast&mask)/mask),axis = -1)).astype(int)
    broadcast[broadcast&mask==0] = 0
    broadcast = (np.bitwise_or.reduce(broadcast,axis = -1)&0x1FFF)
    score = np.max((broadcast|(suitnum<<13)).T,axis = -1)
    return score

def playerhandeval(yourhand):
    yourhand = np.swapaxes(yourhand.reshape(-1,6,7),0,1)
    mask = np.array([0x2000,0x4000,0x8000,0x10000]).reshape(4,1,1,1)
    broadcast = np.copy(np.tile(yourhand,(4,1,1,1)))
    suitnum = (np.sum((( broadcast&mask)/mask),axis = -1)).astype(int)
    broadcast[broadcast&mask==0] = 0
    broadcast = (np.bitwise_or.reduce(broadcast,axis = -1)&0x1FFF)
    score = np.max((broadcast|(suitnum<<13)).T,axis = -1)
    return score


def getreturn(yourhand,dealerhand):
    start = time.process_time()
    dealereval = np.tile(handeval(dealerhand),(6,1)).T
#     start = time.process_time()
#     player1eval = handeval(yourhand[:,0:7])
#     player2eval = handeval(yourhand[:,7:14])
#     player3eval = handeval(yourhand[:,14:21])
#     player4eval = handeval(yourhand[:,21:28])
#     player5eval = handeval(yourhand[:,28:35])
#     player6eval = handeval(yourhand[:,35:42])
#     print(time.process_time() - start)
#     playereval = np.stack((player1eval,player2eval,player3eval,player4eval,player5eval,player6eval))
    playereval = playerhandeval(yourhand)
    unitreturn = np.ones_like(playereval)
    unitreturn *= 2
    unitreturn[playereval>=0xa000] = 3
    unitreturn[playereval>=0xc000] = 4
    unitreturn[playereval<dealereval] *= -1
    unitreturn[playereval==dealereval] = 0
    unitreturn[dealereval<0x6100] = 1
    return np.mean(unitreturn)

def getreturn1(yourhand,dealerhand):
    start = time.process_time()
    dealereval = np.tile(handeval(dealerhand),(6,1))
    player1eval = handeval(yourhand[:,0:7])
    player2eval = handeval(yourhand[:,7:14])
    player3eval = handeval(yourhand[:,14:21])
    player4eval = handeval(yourhand[:,21:28])
    player5eval = handeval(yourhand[:,28:35])
    player6eval = handeval(yourhand[:,35:42])
    playereval = np.stack((player1eval,player2eval,player3eval,player4eval,player5eval,player6eval))
    unitreturn = np.ones_like(playereval)
    unitreturn *= 2
    unitreturn[playereval>=0xa000] = 3
    unitreturn[playereval>=0xc000] = 4
    unitreturn[playereval<dealereval] *= -1
    unitreturn[playereval==dealereval] = 0
    unitreturn[dealereval<0x6100] = 1
    return np.mean(unitreturn)

In [3]:
x0 = np.array([1<<i for i in range(13)])
x1 = (1<<13)+x0
x2 = (2<<13)+x0
x4 = (4<<13)+x0
x8 = (8<<13)+x0
bit = np.concatenate((x1,x2,x4,x8))
bit = np.tile(bit,(1000000,1))

In [4]:
for _ in range(1):
    start = time.process_time()
    rng = np.random.default_rng()
    bit = rng.permuted(bit, axis=1)
    cProfile.run('foo()')
    getreturn(bit[:,7:49],bit[:,0:7])
    print(time.process_time() - start)


0.6453389999999999
(1000000, 6)
(1000000, 6)
-0.09949166666666667
7.931177999999999


In [13]:
def run():
    x0 = np.array([1<<i for i in range(13)])
    x1 = (1<<13)+x0
    x2 = (2<<13)+x0
    x4 = (4<<13)+x0
    x8 = (8<<13)+x0
    bit = np.concatenate((x1,x2,x4,x8))
    bit = np.tile(bit,(1000000,1))

    rng = np.random.default_rng()
    bit = rng.permuted(bit, axis=1)
    return getreturn1(bit[:,7:49],bit[:,0:7])

In [17]:
cProfile.run('run()')

         413 function calls (410 primitive calls) in 6.630 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.010    0.010    6.621    6.621 4206590963.py:1(run)
        1    0.000    0.000    0.000    0.000 4206590963.py:2(<listcomp>)
        7    2.834    0.405    5.220    0.746 585250354.py:1(handeval)
        1    0.113    0.113    5.362    5.362 585250354.py:44(getreturn1)
        7    0.000    0.000    0.024    0.003 <__array_function__ internals>:2(amax)
        3    0.000    0.000    0.009    0.003 <__array_function__ internals>:2(concatenate)
        7    0.000    0.000    0.453    0.065 <__array_function__ internals>:2(copy)
        1    0.000    0.000    0.010    0.010 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(empty_like)
        1    0.000    0.000    0.003    0.003 <__array_function__ internals>:2(mean)
        1    0.000 